In [1]:
import pickle # 데이터 불러오는 용도 (벡터 리스트, 전처리 자료들 불러오는 용도)
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

In [2]:
# 벡터 리스트 불러오기
device = "cuda" if torch.cuda.is_available() else "cpu"

with open("sound_vector_list.pkl", mode = "rb") as f:
    vectors = torch.tensor(pickle.load(f), dtype = torch.float, device = device)
with open("sound_dict.pkl", mode = "rb") as f:
    sound_dict = pickle.load(f)

print(vectors.shape)
pad_idx = sound_dict["<pad>"]
sos_idx = sound_dict["<sos>"]
eos_idx = sound_dict["<eos>"]

torch.Size([11181, 76])


In [3]:
# 데이터 불러오기

def dataload(x_file_name, t_file_name, batch_size, shuffle, device) :
    with open(x_file_name, mode = "rb") as f:
        tensor_x = torch.tensor(pickle.load(f), dtype = torch.float, device = device)
        print(tensor_x.shape)
    with open(t_file_name, mode = "rb") as f:
        tensor_t = torch.tensor(pickle.load(f), dtype = torch.long, device = device)
        print(tensor_t.shape)
    return DataLoader(list(zip(tensor_x, tensor_t)), batch_size=batch_size, shuffle=shuffle)

train_dataloader = dataload("data/sound_data_train.pkl", "data/text_data_train.pkl", 100, True, device)
test_dataloader = dataload("data/sound_data_test.pkl", "data/text_data_test.pkl", 100, True, device)

print(len(train_dataloader))
print(len(test_dataloader))

torch.Size([10002, 196, 64])
torch.Size([10002, 6])
torch.Size([1250, 182, 64])
torch.Size([1250, 6])
101
13


In [4]:
# 신경만 미리 만들어 놓은거 불러오기

from NN import Encoder
from NN import Attention
from NN import Decoder

In [5]:

encoder = Encoder(64, vectors.shape[1], num_layers=2, bidirectional=True)
decoder = Decoder(vectors, vectors.shape[1], num_layers=2, bidirectional=True, max_len=6, pad_idx=pad_idx, sos_idx=sos_idx)